Build a GA that constructs exam or class timetables under realistic constraints.
For the project, choose one small or medium-sized instance (from Toronto or ITC 2007) and
implement at least:
● all hard constraints (no student in two exams at the same time),
● one or two soft criteria in your fitness function.

By 10 December you should submit a complete mini-project:
A working GA implementation for your chosen problem and dataset.

- At least a few experiments:  
   -  different parameter settings and/or different fitness/operators,  
   -  several runs per configuration, with basic metrics (best/average quality, etc.).  

- clean repository:  
   - readable code  
   - clear instructions in README (how to run, what to expect),  
   - a short summary of results and main conclusions.  

I'm going to use Toronto dataset

In [79]:
import random
from typing import List, Tuple

# For reproducibility
random.seed(42)

test_dataset = "datasets/uta-s-92-2"
exam_dataset_path = test_dataset + ".crs"
students_dataset_path = test_dataset + ".stu"

Representation format: Array of N elements, where N - number of exams.  
So genom always contains all and indeed valid.

In [80]:
my_candidate = [0, 1, 2, 0, 3, 2]

In [81]:
def read_exams(path: str) -> List[int]:
    f = open(path, "r")
    frl = f.readlines()
    exams = [int(el.split()[0]) for el in frl]
    return exams

def read_students(path: str) -> List[List[int]]:
    f = open(path, "r")
    frl = f.readlines()
    students = list(map(lambda x: list(map(int, x.split())), frl))
    return students

exams = read_exams(exam_dataset_path)
students = read_students(students_dataset_path)

number_of_exams = len(exams)
number_of_students = len(students)

c = [[0 for _ in range(number_of_exams + 1)] for _ in range(number_of_exams + 1)]

def intersect_exams(students: List[List[int]])-> List[List[int]]:
    for student in students:
        for a in student:
            for b in student:
                if a != b:
                    c[a][b] += 1

intersect_exams(students)

In [82]:
from math import comb

def fitness(individual):
    fit = 0

    for i in range(number_of_exams):
        slot_i = individual[i]

        for j in range(i + 1, number_of_exams):
            slot_j = individual[j]

            conflict = c[i][j]
            if conflict == 0:
                continue

            if slot_i == slot_j:
                fit -= 10000 * conflict
                continue

            if abs(slot_i - slot_j) >= 5:
                continue

            dt = abs(slot_i - slot_j)

            w = 2 ** max(0, 4 - dt)
            fit -= w * conflict

    return fit


In [83]:
my_candidate = list(random.randint(0, 5) for _ in range(number_of_exams))
fit_value = fitness(my_candidate)
print(f"Fitness of candidate {my_candidate} is {fit_value}")

Fitness of candidate [5, 0, 0, 5, 2, 1, 1, 1, 5, 0, 5, 5, 4, 0, 4, 3, 0, 0, 0, 1, 1, 4, 4, 0, 4, 1, 5, 5, 5, 4, 3, 1, 3, 4, 2, 0, 1, 5, 3, 2, 2, 1, 1, 2, 0, 0, 3, 0, 2, 2, 4, 2, 0, 5, 3, 4, 0, 3, 0, 4, 2, 5, 4, 2, 4, 1, 5, 0, 0, 5, 1, 2, 0, 1, 0, 3, 2, 3, 5, 2, 1, 2, 2, 1, 5, 2, 5, 5, 5, 0, 4, 5, 1, 4, 5, 1, 1, 3, 3, 2, 5, 5, 4, 1, 5, 2, 0, 1, 0, 2, 3, 2, 0, 1, 4, 5, 2, 1, 5, 3, 3, 5, 3, 1, 2, 1, 1, 5, 4, 4, 2, 5, 4, 3, 4, 3, 2, 1, 1, 4, 3, 0, 0, 0, 1, 5, 1, 5, 3, 4, 0, 3, 3, 4, 3, 4, 2, 4, 0, 5, 5, 0, 5, 4, 2, 5, 2, 0, 2, 3, 1, 3, 0, 5, 5, 2, 4, 1, 4, 0, 5, 2, 5, 4, 4, 1, 1, 2, 1, 4, 4, 0, 4, 2, 3, 0, 0, 2, 2, 1, 0, 1, 4, 0, 0, 5, 3, 0, 4, 1, 1, 5, 3, 4, 1, 2, 4, 4, 3, 1, 4, 5, 5, 1, 5, 2, 3, 5, 5, 2, 3, 4, 3, 0, 1, 1, 0, 2, 0, 4, 4, 1, 4, 1, 0, 0, 5, 5, 0, 1, 0, 0, 2, 0, 4, 1, 2, 5, 3, 1, 4, 1, 5, 4, 4, 3, 1, 3, 3, 1, 0, 0, 5, 3, 2, 3, 3, 3, 5, 0, 5, 5, 5, 0, 0, 3, 5, 2, 0, 1, 1, 1, 4, 3, 1, 3, 1, 2, 3, 1, 0, 3, 4, 0, 0, 5, 4, 0, 0, 1, 1, 3, 3, 3, 1, 3, 0, 1, 3, 0, 3, 2, 3, 2, 3, 5, 

### Tournament

In [84]:
def create_initial_population(pop_size: int,
                              exam_count: int,
                              max_time_slot: int = 50) -> List[List[int]]:
    """Create an initial population of random individuals."""
    population = []
    for _ in range(pop_size):
        individual = [random.randint(0, max_time_slot - 1)
                      for _ in range(exam_count)]
        population.append(individual)
    return population

def tournament_selection(population: List[List[int]],
                         fitnesses: List[int],
                         k: int = 3) -> List[int]:
    """Select one parent using tournament selection."""
    idxs = random.sample(list(range(len(population))), k)
    fitnesses = [(fitnesses[i], i) for i in idxs]
    return population[max(fitnesses)[1]]
population = create_initial_population(10, number_of_exams)

### Crossover

In [85]:
from typing import Tuple, List
import random

def one_point_crossover(parent1: List[int],
                        parent2: List[int],
                        crossover_prob: float = 0.8) -> Tuple[List[int], List[int]]:
    """
    Perform one-point crossover with given probability.
    """
    n = number_of_exams

    r = random.random()
    if r > crossover_prob:
      return (parent1, parent2)
    c = random.randint(1, n-2)
    child1 = parent1[:c] + parent2[c:]
    child2 = parent2[:c] + parent1[c:]

    return (child1, child2)

### Mutation

In [86]:
def mutate(individual: List[int],
           mutation_prob: float = 0.1,
           max_time_slots: int = 50) -> None:
    """
    Mutate the individual *in place*.
    """
    n = number_of_exams
    for i in range(n):
      if random.random() < mutation_prob:
        individual[i] = random.randint(0, max_time_slots - 1)

### Main loop

In [89]:
from multiprocessing import Pool

def genetic_algorithm(pop_size: int = 30,
                      max_generations: int = 200,
                      crossover_prob: float = 0.8,
                      mutation_prob: float = 0.1,
                      tournament_k: int = 10,
                      max_time_slots: int = 50,
                      verbose: bool = True,
                      n_jobs: int = None):
    """
    Run the genetic algorithm with parallel fitness evaluation.
    n_jobs = number of workers (None => use all cores)
    """

    population = create_initial_population(pop_size, number_of_exams, max_time_slots)

    best_individual = None
    best_fitness = float("-inf")
    success_generation = None

    # Create worker pool once (not every generation!)
    with Pool(processes=n_jobs) as pool:

        for gen in range(max_generations):

            # ======== PARALLEL FITNESS ==========
            fitnesses = pool.map(fitness, population)
            # =====================================

            best_idx = max(range(len(population)), key=lambda i: fitnesses[i])
            if fitnesses[best_idx] > best_fitness:
                best_fitness = fitnesses[best_idx]
                best_individual = population[best_idx][:]

            if verbose and gen % 10 == 0:
                print(f"Generation {gen:3d}: best fitness = {best_fitness}")

            if best_fitness == 0:
                success_generation = gen
                if verbose:
                    print(f"Solution found at generation {gen}!")
                break

            # ======== CREATE NEXT GENERATION ========
            new_population = []

            while len(new_population) < pop_size:
                parent1 = tournament_selection(population, fitnesses, tournament_k)
                parent2 = tournament_selection(population, fitnesses, tournament_k)

                c1, c2 = one_point_crossover(parent1, parent2, crossover_prob)

                mutate(c1, mutation_prob, max_time_slots)
                mutate(c2, mutation_prob, max_time_slots)

                new_population.append(c1)
                new_population.append(c2)

            population = new_population[:pop_size]
            # ========================================

    return best_individual, best_fitness, success_generation

best, best_f, gen_succ = genetic_algorithm(n_jobs=4)
print("Best fitness:", best_f)
print("Success generation:", gen_succ)

Generation   0: best fitness = -10954906
Generation  10: best fitness = -8454781
Generation  20: best fitness = -8454781
Generation  30: best fitness = -8454781
Generation  40: best fitness = -8454781
Generation  50: best fitness = -8454781
Generation  60: best fitness = -8454781
Generation  70: best fitness = -8454781
Generation  80: best fitness = -8454781
Generation  90: best fitness = -8454781
Generation 100: best fitness = -8454781
Generation 110: best fitness = -8454781
Generation 120: best fitness = -8454781
Generation 130: best fitness = -8454781
Generation 140: best fitness = -8454781
Generation 150: best fitness = -8454781
Generation 160: best fitness = -8074398
Generation 170: best fitness = -8074398
Generation 180: best fitness = -8074398
Generation 190: best fitness = -8074398
Best fitness: -8074398
Success generation: None


### Experiments